In [ ]:
import numpy as np


In [ ]:
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

In [ ]:
print(datain)
print(datain.shape)   # Useful if it’s an array
print(type(datain)) 

In [ ]:
print(dataout)
print(dataout.shape)   # Useful if it’s an array
print(type(dataout)) 

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# -------------------------
# Put your data here
# -------------------------
X_init = datain
y_init = dataout
# -------------------------
# Transform / settings
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_init.shape[1])  # assume normalized [0,1] per dim
dim = X_init.shape[1]
y_best = y_init.max()

# -------------------------
# Fit GP surrogate
# -------------------------
kernel = Matern(length_scale=np.ones(dim),
                length_scale_bounds=(1e-2, 1e2),
                nu=2.5)
# alpha = noise variance. If outputs are noisy, increase alpha (e.g. 1e-4 or tuned)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True, n_restarts_optimizer=5, random_state=0)
gp.fit(X_init, y_init)

# -------------------------
# Expected Improvement (EI)
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei  # note: maximize this

# Negative EI for minimizers
def neg_ei(x, gp, y_best, xi=0.01):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi=xi)[0]

# -------------------------
# Acquisition optimization: many random starts + DE fallback
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40):
    dim = bounds.shape[0]
    best_x = None
    best_val = 1e20

    # Random-start L-BFGS-B
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun=neg_ei,
                       x0=x0,
                       args=(gp, y_best),
                       bounds=bounds,
                       method='L-BFGS-B',
                       options={'maxiter':200})
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    # Try differential evolution as a global check (optional, slower)
    try:
        de_res = differential_evolution(lambda x: neg_ei(x, gp, y_best),
                                       bounds=bounds.tolist(),
                                       maxiter=200, polish=True, seed=0)
        if de_res.fun < best_val:
            best_val = de_res.fun
            best_x = de_res.x
    except Exception:
        pass

    # clamp to bounds and return
    best_x = np.clip(best_x, bounds[:,0], bounds[:,1])
    return best_x

# -------------------------
# If you need a small batch (greedy sequential EI)
# -------------------------
def propose_batch(gp, y_best, bounds, batch_size=3):
    batch = []
    gp_copy = gp
    X_aug = X_init.copy()
    y_aug = y_init.copy()
    for i in range(batch_size):
        x_next = propose_location(gp_copy, y_aug.max(), bounds)
        # "Fake" evaluation: use GP predicted mean (Kriging believer)
        y_fake = gp_copy.predict(x_next.reshape(1, -1))[0]
        # Append to augmented set and refit GP (greedy sequential)
        X_aug = np.vstack([X_aug, x_next])
        y_aug = np.hstack([y_aug, y_fake])
        gp_copy = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
        gp_copy.fit(X_aug, y_aug)
        batch.append(x_next)
    return np.array(batch)

# -------------------------
# Run proposal
# -------------------------
x_next = propose_location(gp, y_best, np.array(bounds))
print("Suggested next point (continuous):", x_next)

# If some dims are categorical/encoded, round or map them here, e.g.:
# x_next[cat_index] = int(np.round(x_next[cat_index] * (num_categories-1)))
#
# Example: If dim 6 encodes optimizer 0..3:
# x_next[6] = int(np.round(x_next[6] * 3))

# -------------------------
# (Optional) propose a small batch
# -------------------------
batch = propose_batch(gp, y_best, np.array(bounds), batch_size=3)
print("Suggested batch of 3 points (continuous):\n", batch)


In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern
from scipy.optimize import minimize
from scipy.stats import norm

# --- Initial data ---
#X_init = np.array([...])  # 30x6 array
#y_init = np.array([...])  # 30 outputs
X_init = datain
y_init = dataout

y_best = y_init.max()  # best performance so far

# --- Fit Gaussian Process ---
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_init, y_init)

# --- Expected Improvement acquisition function ---
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.array(x).reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    mu, sigma = mu[0], sigma[0]
    if sigma == 0.0:
        return 0.0
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return -ei  # negative for minimization in scipy

# --- Optimize acquisition function ---
bounds = [(0,1)]*6
best_x = None
best_ei = float('inf')

for _ in range(20):  # multiple random starts
    x0 = np.random.rand(6)
    res = minimize(lambda x: expected_improvement(x, gp, y_best),
                   x0=x0, bounds=bounds, method='L-BFGS-B')
    if res.fun < best_ei:
        best_ei = res.fun
        best_x = res.x

x_next = best_x
print("Next hyperparameter set to try:", x_next)


# Week 2

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
# -------------------------
# Existing data (your previous iteration)
# -------------------------
X_init = datain
y_init = dataout

# Latest evaluated point (the one that gave 9.68029...)
X_new = np.array([[0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362]])
y_new = np.array([9.6802928063541])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

# -------------------------
# Step 2: Fit updated GP surrogate
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim),
                length_scale_bounds=(1e-2, 1e2),
                nu=2.5)

# Lower alpha for smooth deterministic behavior
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6,
                              normalize_y=True, n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

# -------------------------
# Step 3: Expected Improvement (EI) — exploitative mode
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):  # smaller xi = exploit
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi=xi)[0]

# -------------------------
# Step 4: Optimize acquisition to find next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40):
    best_x, best_val = None, 1e20
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun=neg_ei,
                       x0=x0,
                       args=(gp, y_best, 0.001),
                       bounds=bounds,
                       method='L-BFGS-B',
                       options={'maxiter':200})
        if res.fun < best_val:
            best_val, best_x = res.fun, res.x

    # Optional global search (safety)
    try:
        de_res = differential_evolution(lambda x: neg_ei(x, gp, y_best, 0.001),
                                        bounds=bounds.tolist(),
                                        maxiter=200, polish=True, seed=0)
        if de_res.fun < best_val:
            best_val, best_x = de_res.fun, de_res.x
    except Exception:
        pass

    return np.clip(best_x, bounds[:,0], bounds[:,1])

# Compute updated best observed value
y_best = y_all.max()

# Get next suggested point
x_next = propose_location(gp, y_best, bounds)

print("Next suggested point (exploit mode):", x_next)


# Week 3

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
# -------------------------
# Existing data (your previous iteration)
# -------------------------
X_init = datain
y_init = dataout

In [ ]:
# Latest evaluated point (the one that gave 9.68029...) from Week 1
X_new = np.array([[0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362]])
y_new = np.array([9.6802928063541])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

In [ ]:
# Latest evaluated point (the one that gave 9.62007...) from Week 2
X_new = np.array([[0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999]])
y_new = np.array([9.6200730832974])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

In [ ]:
print(X_all)
print(X_all.shape)   # Useful if it’s an array
print(type(X_all)) 
print("-----------------------------------------------")
print(y_all)
print(y_all.shape)   # Useful if it’s an array
print(type(y_all)) 

In [ ]:
# -------------------------
# Step 2: Fit updated GP surrogate
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim),
                length_scale_bounds=(1e-2, 1e2),
                nu=2.5)

# Lower alpha for smooth deterministic behavior
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6,
                              normalize_y=True, n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

# -------------------------
# Step 3: Expected Improvement (EI) — exploitative mode
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):  # smaller xi = exploit
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi=xi)[0]

# -------------------------
# Step 4: Optimize acquisition to find next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40):
    best_x, best_val = None, 1e20
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun=neg_ei,
                       x0=x0,
                       args=(gp, y_best, 0.001),
                       bounds=bounds,
                       method='L-BFGS-B',
                       options={'maxiter':200})
        if res.fun < best_val:
            best_val, best_x = res.fun, res.x

    # Optional global search (safety)
    try:
        de_res = differential_evolution(lambda x: neg_ei(x, gp, y_best, 0.001),
                                        bounds=bounds.tolist(),
                                        maxiter=200, polish=True, seed=0)
        if de_res.fun < best_val:
            best_val, best_x = de_res.fun, de_res.x
    except Exception:
        pass

    return np.clip(best_x, bounds[:,0], bounds[:,1])

# Compute updated best observed value
y_best = y_all.max()

# Get next suggested point
x_next = propose_location(gp, y_best, bounds)

print("Next suggested point (exploit mode):", x_next)

x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded:", x_next_6dp)


# Slight tweak to slightly shrink the search bounds to avoid exact 0 or 1 values,
# without adding any penalties or acquisition modifications.

In [ ]:
# updated propose_location() function with the soft clamping fix integrated.
# This will stop 0.0 and 1.0 values, while leaving everything else about your Bayesian optimization loop unchanged.

In [ ]:
# -------------------------
# Step 4: Optimize EI to get next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40, xi=0.001):
    dim = bounds.shape[0]
    best_x = None
    best_val = 1e20

    # Random-start L-BFGS-B local optimization
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1])
        res = minimize(
            fun=neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method='L-BFGS-B',
            options={'maxiter': 200}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    # Global backup using Differential Evolution
    try:
        de_res = differential_evolution(
            lambda x: neg_ei(x, gp, y_best, xi),
            bounds=bounds.tolist(),
            maxiter=200,
            polish=True,
            seed=0
        )
        if de_res.fun < best_val:
            best_val = de_res.fun
            best_x = de_res.x
    except Exception:
        pass

    # --------------------------------------
    # Soft clamp to avoid hitting exact edges
    # --------------------------------------
    epsilon = 0.01  # safety margin
    best_x = np.clip(best_x, epsilon, 1 - epsilon)

    return best_x


In [ ]:
# -------------------------
# Step 2: Fit updated GP surrogate
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim),
                length_scale_bounds=(1e-2, 1e2),
                nu=2.5)

# Lower alpha for smooth deterministic behavior
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6,
                              normalize_y=True, n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

# -------------------------
# Step 3: Expected Improvement (EI) — exploitative mode
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):  # smaller xi = exploit
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi=xi)[0]

# -------------------------
# Step 4: Optimize acquisition to find next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40):
    best_x, best_val = None, 1e20
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun=neg_ei,
                       x0=x0,
                       args=(gp, y_best, 0.001),
                       bounds=bounds,
                       method='L-BFGS-B',
                       options={'maxiter':200})
        if res.fun < best_val:
            best_val, best_x = res.fun, res.x

    # Optional global search (safety)
    try:
        de_res = differential_evolution(lambda x: neg_ei(x, gp, y_best, 0.001),
                                        bounds=bounds.tolist(),
                                        maxiter=200, polish=True, seed=0)
        if de_res.fun < best_val:
            best_val, best_x = de_res.fun, de_res.x
    except Exception:
        pass

    return np.clip(best_x, bounds[:,0], bounds[:,1])

# Compute updated best observed value
y_best = y_all.max()

# Get next suggested point
x_next = propose_location(gp, y_best, bounds)

print("Next suggested point (exploit mode):", x_next)

x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded:", x_next_6dp)


In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded to:", x_next_6dp)

# Week 4

In [ ]:
#Note: ran Week 3 with missing data accidentally (41 inputs not 42)


Previous high outputs: 9.6803, 9.6201
This new point improves the current best: y_best = 9.7668
The point is near previous high regions, especially in dimensions 4, 5, and 8 which are now close to the edges (0.01 / 0.99 after clamping).

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')
# -------------------------
# Existing data (your previous iteration)
# -------------------------
X_init = datain
y_init = dataout

In [ ]:
# Latest evaluated point (the one that gave 9.68029...) from Week 1
X_new = np.array([[0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362]])
y_new = np.array([9.6802928063541])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_init, X_new])
y_all = np.concatenate([y_init, y_new])

In [ ]:
# Latest evaluated point (the one that gave 9.62007...) from Week 2
X_new = np.array([[0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999]])
y_new = np.array([9.6200730832974])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_all, X_new])
y_all = np.concatenate([y_all, y_new])

In [ ]:
# Latest evaluated point (the one that gave 9.766762...) from Week 3
X_new = np.array([[0.251068, 0.078758, 0.302102, 0.010000, 0.990000, 0.357561, 0.178631, 0.010000]])
y_new = np.array([9.766762063933])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_all, X_new])
y_all = np.concatenate([y_all, y_new])

In [ ]:
print(X_all)
print(X_all.shape)   # Useful if it’s an array
print(type(X_all)) 
print("-----------------------------------------------")
print(y_all)
print(y_all.shape)   # Useful if it’s an array
print(type(y_all)) 

# Summary
The new output confirms a rising trend toward a local maximum.
Strategy: continue local exploitation near the current best.
Maintain safety margins (clamped bounds) and low xi.
Each iteration: append new (X, y), refit GP, maximize EI → next point.

In [ ]:
# -------------------------
# Step 4: Optimize EI to get next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40, xi=0.001):
    dim = bounds.shape[0]
    best_x = None
    best_val = 1e20

    # Random-start L-BFGS-B local optimization
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1])
        res = minimize(
            fun=neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method='L-BFGS-B',
            options={'maxiter': 200}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    # Global backup using Differential Evolution
    try:
        de_res = differential_evolution(
            lambda x: neg_ei(x, gp, y_best, xi),
            bounds=bounds.tolist(),
            maxiter=200,
            polish=True,
            seed=0
        )
        if de_res.fun < best_val:
            best_val = de_res.fun
            best_x = de_res.x
    except Exception:
        pass

    # --------------------------------------
    # Soft clamp to avoid hitting exact edges
    # --------------------------------------
    epsilon = 0.01  # safety margin
    best_x = np.clip(best_x, epsilon, 1 - epsilon)

    return best_x


In [ ]:
# -------------------------
# Step 2: Fit Gaussian Process
# -------------------------
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim), 
                length_scale_bounds=(1e-2, 1e2), 
                nu=2.5)

gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

# -------------------------
# Step 3: Expected Improvement (EI)
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x, gp, y_best, xi)

# -------------------------
# Step 4: Optimize EI to get next candidate (propose location above)
# -------------------------


# -------------------------
# Step 5: Get next point (exploitative mode)
# -------------------------
y_best = y_all.max()
x_next = propose_location(gp, y_best, bounds, n_restarts=40, xi=0.001)

print("Next suggested point (exploit mode, safe bounds):", x_next)

x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded to:", x_next_6dp)

# Week 5

In [ ]:
# assume previous additions have been executed
# Latest evaluated point (the one that gave 9.37782...) from Week 4
X_new = np.array([[0.122075, 0.010000, 0.420807, 0.010000, 0.839488, 0.990000, 0.010000, 0.990000]])
y_new = np.array([9.377829603931])

# -------------------------
# Step 1: Append new data
# -------------------------
X_all = np.vstack([X_all, X_new])
y_all = np.concatenate([y_all, y_new])

This new point produced a lower output (9.3778) than your current best (9.7668). It is not near the current best in input space (Euclidean distance ≈ 1.20) — in particular it flips several coordinates relative to the best (most notably dimension 8 and 6). So this evaluation is negative evidence for that region: it helps the GP learn the shape of the surface (it reduces uncertainty and lowers predicted mean where those coordinate changes occur). The practical consequence is: don’t switch strategy — append & refit the GP, then continue exploitative local refinement around the true best, while (optionally) doing a small amount of targeted exploration to test sensitive dimensions.

- Current best (so far): X_best ≈ [0.251068, 0.078758, 0.302102, 0.01, 0.99, 0.357561, 0.178631, 0.01] → y_best = 9.766762063933
- New point: X_new = [0.122075, 0.01, 0.420807, 0.01, 0.839488, 0.99, 0.01, 0.99] → y_new = 9.377829603931
- the biggest flips (dim 8 and dim 6) correlate with a drop in performance — a sign those coordinates are sensitive and likely important to the local optimum.

In [ ]:
# -------------------------
# Step 4: Optimize EI to get next candidate
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=40, xi=0.001):
    dim = bounds.shape[0]
    best_x = None
    best_val = 1e20

    # Random-start L-BFGS-B local optimization
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1])
        res = minimize(
            fun=neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method='L-BFGS-B',
            options={'maxiter': 200}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    # Global backup using Differential Evolution
    try:
        de_res = differential_evolution(
            lambda x: neg_ei(x, gp, y_best, xi),
            bounds=bounds.tolist(),
            maxiter=200,
            polish=True,
            seed=0
        )
        if de_res.fun < best_val:
            best_val = de_res.fun
            best_x = de_res.x
    except Exception:
        pass

    # --------------------------------------
    # Soft clamp to avoid hitting exact edges
    # --------------------------------------
    epsilon = 0.01  # safety margin
    best_x = np.clip(best_x, epsilon, 1 - epsilon)

    return best_x

def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x, gp, y_best, xi)

In [ ]:
bounds = np.array([(0.0, 1.0)] * X_all.shape[1])
dim = X_all.shape[1]

kernel = Matern(length_scale=np.ones(dim), 
                length_scale_bounds=(1e-2, 1e2), 
                nu=2.5)

gp = GaussianProcessRegressor(kernel=kernel,
                              alpha=1e-6,
                              normalize_y=True,
                              n_restarts_optimizer=5,
                              random_state=0)
gp.fit(X_all, y_all)

y_best = y_all.max()
x_best = X_all[np.argmax(y_all)]

append point, refit GP, then propose next point using a local search around current best

In [ ]:
import numpy as np

# Build local bounds around x_best
local_radius = 0.12  # change 0.08-0.15 to tune locality
eps = 1e-3
dim = X_all.shape[1]
local_bounds = []
for i in range(dim):
    low = max(eps, x_best[i] - local_radius)
    high = min(1 - eps, x_best[i] + local_radius)
    # If a coordinate is already very tight (e.g. near 0.01/0.99), keep it tighter:
    if x_best[i] <= 0.02 or x_best[i] >= 0.98:
        # keep small wiggle
        low = max(eps, x_best[i] - 0.02)
        high = min(1 - eps, x_best[i] + 0.02)
    local_bounds.append((low, high))
local_bounds = np.array(local_bounds)

# Propose next point (exploitative EI)
x_next = propose_location(gp, y_best, local_bounds, n_restarts=20, xi=0.001)

print("x_best (current):", x_best)
print("y_best:", y_best)
print("Next candidate (local exploit):", x_next)


In [ ]:
x_next_6dp = np.round(x_next, 6)
x_next_6dp
print("Rounded to:", x_next_6dp)

# Week 6
- good news: the new evaluation is better than the previous best and it validates the direction of the local search.
- New point: [0.214679, 0.179152, 0.182102, 0.01, 0.99, 0.278369, 0.298631, 0.01]
- New output: 9.823667311119 → new best (previous best was 9.766762063933)
- further confirmation of a local optimum region — the search is correctly refining the peak rather than wandering off.

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# ------------------------------------------------------------
# Load original data
# ------------------------------------------------------------
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

X_all = datain.copy()
y_all = dataout.copy()

# ------------------------------------------------------------
# Append new evaluations (in chronological order)
# ------------------------------------------------------------

new_points = [
    ([0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362], 9.6802928063541),
    ([0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999], 9.6200730832974),
    ([0.251068, 0.078758, 0.302102, 0.010000, 0.990000, 0.357561, 0.178631, 0.010000], 9.766762063933),
    ([0.122075, 0.010000, 0.420807, 0.010000, 0.839488, 0.990000, 0.010000, 0.990000], 9.377829603931),
    ([0.214679, 0.179152, 0.182102, 0.010000, 0.990000, 0.278369, 0.298631, 0.010000], 9.823667311119),
]

for x, y in new_points:
    X_all = np.vstack([X_all, np.array(x)])
    y_all = np.append(y_all, y)

print("Initialisation complete.")
print("X_all shape:", X_all.shape)
print("y_all shape:", y_all.shape)


In [ ]:
print(X_all)
print(y_all)

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C
from scipy.optimize import minimize

# ------------------------------------------------------------
# 1) Previous four evaluated points and outputs
# ------------------------------------------------------------

# ------------------------------------------------------------
# 2) Fit GP surrogate (GP hyperparameters auto-tuned at fit)
# ------------------------------------------------------------
kernel = C(1.0, (1e-3, 1e3)) * Matern(nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True)
gp.fit(X_all, y_all)

# ------------------------------------------------------------
# 3) Expected Improvement (EI)
# ------------------------------------------------------------
def expected_improvement(x, gp, y_best, xi=0.001):
    x = x.reshape(1, -1)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = sigma.reshape(-1) + 1e-9
    Z = (mu - y_best - xi) / sigma
    from scipy.stats import norm
    return (mu - y_best - xi) * norm.cdf(Z) + sigma * norm.pdf(Z)

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x, gp, y_best, xi)

# ------------------------------------------------------------
# 4) Acquisition optimizer (multi-start)
# ------------------------------------------------------------
def propose_location(gp, y_best, bounds, n_restarts=30, xi=0.001):
    best_x = None
    best_acq = np.inf
    dim = bounds.shape[0]

    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1])
        res = minimize(
            neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method="L-BFGS-B"
        )
        if res.fun < best_acq:
            best_acq = res.fun
            best_x = res.x
    return best_x

# ------------------------------------------------------------
# 5) Build local bounds around the current best
# ------------------------------------------------------------
y_best = y_all.max()
x_best = X_all[np.argmax(y_all)]

local_radius = 0.08
eps = 1e-3
bounds = []

for val in x_best:
    low = max(eps, val - local_radius)
    high = min(1 - eps, val + local_radius)
    bounds.append((low, high))

bounds = np.array(bounds)

# ------------------------------------------------------------
# 6) Propose next candidate (exploitation mode)
# ------------------------------------------------------------
x_next = propose_location(gp, y_best, bounds, n_restarts=30, xi=0.001)

print("Current best point:", x_best)
print("Current best value:", y_best)
print("Next candidate to evaluate:", x_next)


In [ ]:
x_next_6dp = np.round(x_next, 6)
print("Rounded to:", x_next_6dp)

# Week 7
- New point: [0.134679, 0.160603, 0.102102, 0.090000, 0.910000, 0.347374, 0.218631, 0.090000]
- New output: 9.935498490899 → new best so far (previous best was 9.823667311119)
- this suggests the true optimum region may be slightly offset from the previous local peak, and the GP exploitation strategy is correctly honing in on it.

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# -------------------------
# Initial dataset
# -------------------------
datain = np.load('initial_inputs.npy')
dataout = np.load('initial_outputs.npy')

X_all = datain.copy()
y_all = dataout.copy()

# -------------------------
# Previous submissions
# -------------------------
previous_points = np.array([
    [0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362],
    [0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999],
    [0.251068, 0.078758, 0.302102, 0.010000, 0.990000, 0.357561, 0.178631, 0.010000],
    [0.122075, 0.010000, 0.420807, 0.010000, 0.839488, 0.990000, 0.010000, 0.990000],
    [0.214679, 0.179152, 0.182102, 0.010000, 0.990000, 0.278369, 0.298631, 0.010000],
    [0.134679, 0.160603, 0.102102, 0.090000, 0.910000, 0.347374, 0.218631, 0.090000]
])

previous_outputs = np.array([
    9.6802928063541,
    9.6200730832974,
    9.766762063933,
    9.377829603931,
    9.823667311119,
    9.935498490899
])

# Append previous submissions
X_all = np.vstack([X_all, previous_points])
y_all = np.concatenate([y_all, previous_outputs])

- The optimisation strategy is working hierarchically:
- Base-level parameters set the region,
- Higher-level parameters refine the optimum.
- The latest result shows the true peak may be slightly shifted from prior corner-edge assumptions, which is why local refinement is crucial.
- The GP is learning this automatically, and acquisition function (EI) exploitation is producing consistently better points.

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# -------------------------
# Fit GP surrogate
# -------------------------
dim = X_all.shape[1]
kernel = Matern(length_scale=np.ones(dim), length_scale_bounds=(1e-2, 1e2), nu=2.5)
gp = GaussianProcessRegressor(kernel=kernel, alpha=1e-6, normalize_y=True,
                              n_restarts_optimizer=5, random_state=0)
gp.fit(X_all, y_all)
y_best = y_all.max()
x_best = X_all[np.argmax(y_all)]

# -------------------------
# Expected Improvement (EI)
# -------------------------
def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi=xi)[0]

# -------------------------
# Acquisition optimisation
# -------------------------
def propose_location(gp, y_best, bounds, n_restarts=30, xi=0.001):
    best_x = None
    best_val = 1e20

    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun=neg_ei,
                       x0=x0,
                       args=(gp, y_best, xi),
                       bounds=bounds,
                       method='L-BFGS-B',
                       options={'maxiter':200})
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    try:
        de_res = differential_evolution(lambda x: neg_ei(x, gp, y_best, xi),
                                       bounds=bounds.tolist(),
                                       maxiter=200, polish=True, seed=0)
        if de_res.fun < best_val:
            best_val = de_res.fun
            best_x = de_res.x
    except Exception:
        pass

    best_x = np.clip(best_x, bounds[:,0], bounds[:,1])
    return best_x

# -------------------------
# Local bounds for exploitative search
# -------------------------
local_radius = 0.08  # tighten for fine-tuning
eps = 1e-3
local_bounds = []
for i in range(dim):
    if x_best[i] <= 0.02 or x_best[i] >= 0.98:
        # very small wiggle for base-level dims
        low = max(eps, x_best[i] - 0.02)
        high = min(1 - eps, x_best[i] + 0.02)
    else:
        low = max(eps, x_best[i] - local_radius)
        high = min(1 - eps, x_best[i] + local_radius)
    local_bounds.append((low, high))
local_bounds = np.array(local_bounds)

# -------------------------
# Propose next point
# -------------------------
x_next = propose_location(gp, y_best, local_bounds, n_restarts=30, xi=0.001)

print("x_best (current):", x_best)
print("y_best:", y_best)
print("Next candidate (local exploit):", x_next)


In [ ]:
x_next_6dp = np.round(x_next, 6)
print("Rounded to:", x_next_6dp)

# Week 8
- x₄ ≈ 0.01–0.10
- x₅ ≈ 0.90–1.00
- x₈ ≈ 0.01–0.10
- These dimensions have repeatedly locked in at the same region and seem to define the operating regime where high values occur.
The new point continues this pattern almost exactly
- The score (9.8838) is high but slightly below the current best (9.9355)
- This narrows the search towards the sharply peaked maxima instead of encouraging exploration.
- Found a promising peak, but not necessarily the global one.


In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# ---------------------------------------------------
# 1. Load initial dataset
# ---------------------------------------------------
datain = np.load("initial_inputs.npy")
dataout = np.load("initial_outputs.npy")

X_all = datain.copy()
y_all = dataout.copy()

# ---------------------------------------------------
# 2. Append ALL previously evaluated points
# ---------------------------------------------------
new_points = np.array([
    [0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362],
    [0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999],
    [0.251068, 0.078758, 0.302102, 0.010000, 0.990000, 0.357561, 0.178631, 0.010000],
    [0.122075, 0.010000, 0.420807, 0.010000, 0.839488, 0.990000, 0.010000, 0.990000],
    [0.214679, 0.179152, 0.182102, 0.010000, 0.990000, 0.278369, 0.298631, 0.010000],
    [0.134679, 0.160603, 0.102102, 0.090000, 0.910000, 0.347374, 0.218631, 0.090000],
    [0.054679, 0.080603, 0.022102, 0.068388, 0.990000, 0.427374, 0.138631, 0.010000]
])

new_outputs = np.array([
    9.6802928063541,
    9.6200730832974,
    9.766762063933,
    9.377829603931,
    9.823667311119,
    9.935498490899,
    9.883822772355
])

X_all = np.vstack([X_all, new_points])
y_all = np.hstack([y_all, new_outputs])

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# ---------------------------------------------------
# 3. GP model
# ---------------------------------------------------
kernel = Matern(nu=2.5)
gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-6,
    normalize_y=True,
    n_restarts_optimizer=8
)
gp.fit(X_all, y_all)

bounds = np.array([(0.0, 1.0)] * 8)
y_best = y_all.max()

# ---------------------------------------------------
# 4. Expected Improvement
# ---------------------------------------------------
def expected_improvement(x, gp, y_best, xi=0.01):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)

def neg_ei(x, gp, y_best, xi=0.01):
    return -expected_improvement(x, gp, y_best, xi)

# ---------------------------------------------------
# 5. Optimisation helpers
# ---------------------------------------------------
def optimise_acquisition(fun, bounds):
    # Multi-start L-BFGS
    best_x = None
    best_val = 1e20
    for _ in range(40):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(fun, x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x
    return best_x

# ---------------------------------------------------
# 6. Generate two candidates
# ---------------------------------------------------

# ---- A) Exploit (small local jitter around best observed)
best_idx = np.argmax(y_all)
x_best = X_all[best_idx]

local_bounds = np.clip(
    np.vstack([x_best - 0.05, x_best + 0.05]).T,
    0.0, 1.0
)

x_exploit = optimise_acquisition(
    lambda x: neg_ei(x, gp, y_best, xi=0.001),
    local_bounds
)

# ---- B) Explore (full global search)
x_explore = optimise_acquisition(
    lambda x: neg_ei(x, gp, y_best, xi=0.05),
    bounds
)

print("Next EXPLOIT candidate:", x_exploit)
print("Next EXPLORE candidate:", x_explore)


- Going with an EXPLORE  point

In [ ]:
x_next_6dp = np.round(x_explore, 6)
print("Rounded to:", x_next_6dp)

# Week 9
- [0.045771, 0.336234, 0.012812, 0.225992, 0.741620, 0.506746, 0.173188, 0.640432] -> 9.9091106001996
- Previous best was 9.935498490899, so this is slightly lower, but still a high-performing point.
- This point is far from the previous local optimum region in parameter space, particularly in dims 4, 5, 8 (previously ~0.09 / 0.91 / 0.09; now 0.226 / 0.742 / 0.640), and in dims 1–3 and 6–7 as well.
- The point lies outside the previously exploited high-performance region, but still produces a high output.
- This suggests there may be another peak or ridge in the objective landscape that we haven’t fully explored.

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# ---------------------------------------------------
# 1. Load initial dataset
# ---------------------------------------------------
datain = np.load("initial_inputs.npy")
dataout = np.load("initial_outputs.npy")

X_all = datain.copy()
y_all = dataout.copy()

# ---------------------------------------------------
# 2. Append ALL previously evaluated points
# ---------------------------------------------------
new_points = np.array([
    [0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362],
    [0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999],
    [0.251068, 0.078758, 0.302102, 0.010000, 0.990000, 0.357561, 0.178631, 0.010000],
    [0.122075, 0.010000, 0.420807, 0.010000, 0.839488, 0.990000, 0.010000, 0.990000],
    [0.214679, 0.179152, 0.182102, 0.010000, 0.990000, 0.278369, 0.298631, 0.010000],
    [0.134679, 0.160603, 0.102102, 0.090000, 0.910000, 0.347374, 0.218631, 0.090000],
    [0.054679, 0.080603, 0.022102, 0.068388, 0.990000, 0.427374, 0.138631, 0.010000],
    [0.045771, 0.336234, 0.012812, 0.225992, 0.741620, 0.506746, 0.173188, 0.640432]
])

new_outputs = np.array([
    9.6802928063541,
    9.6200730832974,
    9.766762063933,
    9.377829603931,
    9.823667311119,
    9.935498490899,
    9.883822772355,
    9.9091106001996
])

X_all = np.vstack([X_all, new_points])
y_all = np.hstack([y_all, new_outputs])

In [ ]:
# ---------------------------------------------------
# 3. Fit GP surrogate
# ---------------------------------------------------
dim = X_all.shape[1]

kernel = Matern(
    length_scale=np.ones(dim),
    length_scale_bounds=(1e-2, 1e2),
    nu=2.5
)

gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-6,
    normalize_y=True,
    n_restarts_optimizer=5,
    random_state=0
)

gp.fit(X_all, y_all)

# Identify best point seen so far
y_best = y_all.max()
x_best = X_all[np.argmax(y_all)]


# ---------------------------------------------------
# 4. Expected Improvement
# ---------------------------------------------------
from scipy.stats import norm

def expected_improvement(x, gp, y_best, xi=0.001):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    imp = mu - y_best - xi
    Z = imp / sigma
    return imp * norm.cdf(Z) + sigma * norm.pdf(Z)

def neg_ei(x, gp, y_best, xi=0.001):
    return -expected_improvement(x, gp, y_best, xi)


# ---------------------------------------------------
# 5. Acquisition search
# ---------------------------------------------------
def propose_location(bounds, gp, y_best, n_restarts=25, xi=0.001):
    best_val = 1e30
    best_x = None

    # Multi-start local optimisation
    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:,0], bounds[:,1])
        res = minimize(
            fun=neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method='L-BFGS-B',
            options={'maxiter': 200}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    # Differential evolution for global search
    try:
        res_de = differential_evolution(
            lambda x: neg_ei(x, gp, y_best),
            bounds=bounds,
            maxiter=150,
            seed=0,
            polish=True
        )
        if res_de.fun < best_val:
            best_x = res_de.x
    except:
        pass

    return np.clip(best_x, bounds[:,0], bounds[:,1])


# ---------------------------------------------------
# 6. EXploit bounds (local to global best)
# ---------------------------------------------------
eps = 1e-3
exploit_radius = 0.08

exploit_bounds = []
for i in range(dim):
    low = max(eps, x_best[i] - exploit_radius)
    high = min(1 - eps, x_best[i] + exploit_radius)
    exploit_bounds.append((low, high))

exploit_bounds = np.array(exploit_bounds)


# ---------------------------------------------------
# 7. EXplore bounds (local around the new 9.909 point)
# ---------------------------------------------------
x_second_peak = new_points[-1]  # the 9.909 point
explore_radius = 0.15

explore_bounds = []
for i in range(dim):
    low = max(eps, x_second_peak[i] - explore_radius)
    high = min(1 - eps, x_second_peak[i] + explore_radius)
    explore_bounds.append((low, high))

explore_bounds = np.array(explore_bounds)


# ---------------------------------------------------
# 8. Generate candidates
# ---------------------------------------------------
x_next_exploit = propose_location(exploit_bounds, gp, y_best, n_restarts=25, xi=0.001)
x_next_explore  = propose_location(explore_bounds,  gp, y_best, n_restarts=25, xi=0.001)

print("Current best output:", y_best)
print("Current best point:", x_best)
print("\nNext EXPLOIT (local refinement) candidate:")
print(x_next_exploit)
print("\nNext EXPLORE (test second basin) candidate:")
print(x_next_explore)


- Going with an EXPLOIT point

In [ ]:
x_next_6dp = np.round(x_next_exploit, 6)
print("Rounded to:", x_next_6dp)

# Week 10
This is now your highest observed value.
- New output: 9.95459577812
- Previous best: 9.935498490899
- Improvement: +0.0191 → new best
- Stay exploitative for now
- Tight local bounds around the current best
- Goal: see if you can push past ~9.97–9.98

This evaluation confirms that the objective has a broad interior ridge rather than a boundary optimum, and that our Bayesian optimisation strategy has successfully transitioned from peak discovery to ridge refinement, with gains now approaching diminishing returns.

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# ---------------------------------------------------
# 1. Load initial dataset
# ---------------------------------------------------
datain = np.load("initial_inputs.npy")
dataout = np.load("initial_outputs.npy")

X_all = datain.copy()
y_all = dataout.copy()

# ---------------------------------------------------
# 2. Append ALL previously evaluated points
# ---------------------------------------------------
new_points = np.array([
    [0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362],
    [0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999],
    [0.251068, 0.078758, 0.302102, 0.010000, 0.990000, 0.357561, 0.178631, 0.010000],
    [0.122075, 0.010000, 0.420807, 0.010000, 0.839488, 0.990000, 0.010000, 0.990000],
    [0.214679, 0.179152, 0.182102, 0.010000, 0.990000, 0.278369, 0.298631, 0.010000],
    [0.134679, 0.160603, 0.102102, 0.090000, 0.910000, 0.347374, 0.218631, 0.090000],
    [0.054679, 0.080603, 0.022102, 0.068388, 0.990000, 0.427374, 0.138631, 0.010000],
    [0.045771, 0.336234, 0.012812, 0.225992, 0.741620, 0.506746, 0.173188, 0.640432],
    [0.129989, 0.237128, 0.115863, 0.170000, 0.830000, 0.407575, 0.138631, 0.170000]
])

new_outputs = np.array([
    9.6802928063541,
    9.6200730832974,
    9.766762063933,
    9.377829603931,
    9.823667311119,
    9.935498490899,
    9.883822772355,
    9.9091106001996,
    9.95459577812
])

X_all = np.vstack([X_all, new_points])
y_all = np.hstack([y_all, new_outputs])

In [ ]:
# ---------------------------------------------------
# 3. Fit GP surrogate
# ---------------------------------------------------
dim = X_all.shape[1]

kernel = Matern(
    length_scale=np.ones(dim),
    length_scale_bounds=(1e-2, 1e2),
    nu=2.5
)

gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-6,
    normalize_y=True,
    n_restarts_optimizer=5,
    random_state=0
)

gp.fit(X_all, y_all)

y_best = y_all.max()
x_best = X_all[np.argmax(y_all)]

# ---------------------------------------------------
# 4. Expected Improvement
# ---------------------------------------------------
def expected_improvement(x, gp, y_best, xi):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    improvement = mu - y_best - xi
    Z = improvement / sigma
    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi)[0]

# ---------------------------------------------------
# 5. Acquisition optimizer
# ---------------------------------------------------
def propose_location(gp, y_best, bounds, xi, n_restarts=30):
    best_x = None
    best_val = np.inf

    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1])
        res = minimize(
            fun=neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method="L-BFGS-B",
            options={"maxiter": 200}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    return np.clip(best_x, bounds[:, 0], bounds[:, 1])

# ---------------------------------------------------
# 6. EXPLOIT candidate (local, low xi)
# ---------------------------------------------------
local_radius = 0.08
eps = 1e-3

exploit_bounds = []
for i in range(dim):
    if x_best[i] <= 0.02 or x_best[i] >= 0.98:
        low = max(eps, x_best[i] - 0.02)
        high = min(1 - eps, x_best[i] + 0.02)
    else:
        low = max(eps, x_best[i] - local_radius)
        high = min(1 - eps, x_best[i] + local_radius)
    exploit_bounds.append((low, high))

exploit_bounds = np.array(exploit_bounds)

x_exploit = propose_location(
    gp,
    y_best,
    exploit_bounds,
    xi=0.001,
    n_restarts=30
)

# ---------------------------------------------------
# 7. EXPLORE candidate (global, higher xi)
# ---------------------------------------------------
global_bounds = np.array([(0.001, 0.999)] * dim)

x_explore = propose_location(
    gp,
    y_best,
    global_bounds,
    xi=0.1,
    n_restarts=40
)

# ---------------------------------------------------
# 8. Results
# ---------------------------------------------------
print("Current best y:", y_best)
print("Current best x:", x_best)
print("\nNext EXPLOIT candidate:", x_exploit)
print("Next EXPLORE candidate:", x_explore)


# Go with Exploit

In [ ]:
x_next_6dp = np.round(x_exploit, 6)
print("Rounded to:", x_next_6dp)

# Week 11
- Current best ≈ 9.9546 ([0.129989, 0.237128, 0.115863, 0.170000, 0.830000, 0.407575, 0.138631, 0.170000])
- This point ≈ 9.9134 ([0.204502 0.182027 0.035863 0.106562 0.75     0.397148 0.125236 0.25    ])
- Difference ≈ −0.041
- So this is strong but not optimal — it sits just below the current peak.

In [ ]:
import numpy as np
from scipy.optimize import minimize, differential_evolution
from scipy.stats import norm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

# ---------------------------------------------------
# 1. Load initial dataset
# ---------------------------------------------------
datain = np.load("initial_inputs.npy")
dataout = np.load("initial_outputs.npy")

X_all = datain.copy()
y_all = dataout.copy()

# ---------------------------------------------------
# 2. Append ALL previously evaluated points
# ---------------------------------------------------
new_points = np.array([
    [0.203499, 0.167999, 0.195105, 0.035878, 0.999999, 0.999999, 0.224367, 0.498362],
    [0.172222, 0.268299, 0.139005, 0.307855, 0.999999, 0.999999, 0.348896, 0.999999],
    [0.251068, 0.078758, 0.302102, 0.010000, 0.990000, 0.357561, 0.178631, 0.010000],
    [0.122075, 0.010000, 0.420807, 0.010000, 0.839488, 0.990000, 0.010000, 0.990000],
    [0.214679, 0.179152, 0.182102, 0.010000, 0.990000, 0.278369, 0.298631, 0.010000],
    [0.134679, 0.160603, 0.102102, 0.090000, 0.910000, 0.347374, 0.218631, 0.090000],
    [0.054679, 0.080603, 0.022102, 0.068388, 0.990000, 0.427374, 0.138631, 0.010000],
    [0.045771, 0.336234, 0.012812, 0.225992, 0.741620, 0.506746, 0.173188, 0.640432],
    [0.129989, 0.237128, 0.115863, 0.170000, 0.830000, 0.407575, 0.138631, 0.170000],
    [0.204502, 0.182027, 0.035863, 0.106562, 0.750000, 0.397148, 0.125236, 0.250000]
])

new_outputs = np.array([
    9.6802928063541,
    9.6200730832974,
    9.766762063933,
    9.377829603931,
    9.823667311119,
    9.935498490899,
    9.883822772355,
    9.9091106001996,
    9.95459577812,
    9.913402905816
])

X_all = np.vstack([X_all, new_points])
y_all = np.hstack([y_all, new_outputs])

This point lies:

- Between the earlier “edge-heavy” regime (dims near 0.01 / 0.99),
- And the newer interior optimum regime (dims 4, 5, 8 around ~0.1–0.25 / ~0.75–0.9).
Notably:
- Dims 4, 5, 8 are no longer at extremes → confirms that extremes were good but not optimal.
- Dims 1–3, 6–7 remain in the narrow bands you’ve repeatedly seen in high-performing points.
This suggests we are sampling along the same high-value manifold, not discovering a new basin.

In [ ]:
# ---------------------------------------------------
# 3. Fit GP surrogate
# ---------------------------------------------------
dim = X_all.shape[1]

kernel = Matern(
    length_scale=np.ones(dim),
    length_scale_bounds=(1e-2, 1e2),
    nu=2.5
)

gp = GaussianProcessRegressor(
    kernel=kernel,
    alpha=1e-6,
    normalize_y=True,
    n_restarts_optimizer=5,
    random_state=0
)

gp.fit(X_all, y_all)

y_best = y_all.max()
x_best = X_all[np.argmax(y_all)]

# ---------------------------------------------------
# 4. Expected Improvement
# ---------------------------------------------------
def expected_improvement(x, gp, y_best, xi):
    x = np.atleast_2d(x)
    mu, sigma = gp.predict(x, return_std=True)
    sigma = np.maximum(sigma, 1e-12)
    improvement = mu - y_best - xi
    Z = improvement / sigma
    ei = improvement * norm.cdf(Z) + sigma * norm.pdf(Z)
    return ei

def neg_ei(x, gp, y_best, xi):
    return -expected_improvement(x.reshape(1, -1), gp, y_best, xi)[0]

# ---------------------------------------------------
# 5. Acquisition optimizer
# ---------------------------------------------------
def propose_location(gp, y_best, bounds, xi, n_restarts=30):
    best_x = None
    best_val = np.inf

    for _ in range(n_restarts):
        x0 = np.random.uniform(bounds[:, 0], bounds[:, 1])
        res = minimize(
            fun=neg_ei,
            x0=x0,
            args=(gp, y_best, xi),
            bounds=bounds,
            method="L-BFGS-B",
            options={"maxiter": 200}
        )
        if res.fun < best_val:
            best_val = res.fun
            best_x = res.x

    return np.clip(best_x, bounds[:, 0], bounds[:, 1])

# ---------------------------------------------------
# 6. EXPLOIT candidate (local, low xi)
# ---------------------------------------------------
local_radius = 0.08
eps = 1e-3

exploit_bounds = []
for i in range(dim):
    if x_best[i] <= 0.02 or x_best[i] >= 0.98:
        low = max(eps, x_best[i] - 0.02)
        high = min(1 - eps, x_best[i] + 0.02)
    else:
        low = max(eps, x_best[i] - local_radius)
        high = min(1 - eps, x_best[i] + local_radius)
    exploit_bounds.append((low, high))

exploit_bounds = np.array(exploit_bounds)

x_exploit = propose_location(
    gp,
    y_best,
    exploit_bounds,
    xi=0.001,
    n_restarts=30
)

# ---------------------------------------------------
# 7. EXPLORE candidate (global, higher xi)
# ---------------------------------------------------
global_bounds = np.array([(0.001, 0.999)] * dim)

x_explore = propose_location(
    gp,
    y_best,
    global_bounds,
    xi=0.2,
    n_restarts=40
)

# ---------------------------------------------------
# 8. Results
# ---------------------------------------------------
print("Current best y:", y_best)
print("Current best x:", x_best)
print("\nNext EXPLOIT candidate:", x_exploit)
print("Next EXPLORE candidate:", x_explore)


In [ ]:
x_next_6dp = np.round(x_explore, 6)
print("Rounded to:", x_next_6dp)

# Going with EXPLORE 
- repeatedly exploited the same basin (dims ~4, 5, 8 around mid–high values), and exploitation has already delivered strong but plateauing gains (~9.95).
- The recent exploit point (≈9.91) did not improve the best, suggesting diminishing returns in that local region.
- You’ve previously observed secondary high-performing regions (e.g. the 9.90–9.91 points far from the main basin), which means the landscape is likely multi-modal.
- The EXPLORE candidate deliberately pushes into a high-uncertainty region (notably dim 2 ≈ 0.74 and different structure across dims 4–8), where the GP’s uncertainty is still high and upside remains.

# strategy summary
- Earlier rounds: Exploit to lock in a strong local optimum.
- Now: Explore to test whether another peak exists or to rule it out.
- If the explore point underperforms → you can confidently return to exploitation.
- If it performs well → you’ve discovered a new basin worth refining.